In [1]:
#Imports
import pandas as pd
import numpy as np
from fbprophet import Prophet
import logging
import csv
import pickle

In [2]:
#Data
cluster1 = 'C:/Develop/git repositories/datamining2/FBProphet/Clustering/clusters_v1.csv'
cluster2 = 'C:/Develop/git repositories/datamining2/FBProphet/Clustering/clusters_v2.csv'
sales_data = 'C:/Develop/git repositories/datamining2/FBProphet/Clustering/sales.xlsx'
test_data_directory = 'C:/Develop/git repositories/datamining2/data/test/'
output = 'C:/Develop/git repositories/datamining2/FBProphet/Clustering/output'
read_write_prophet_fitted = 'w'
read_write_prophet_forecasts = 'w'

In [3]:
df = pd.read_excel(sales_data)
df_cluster_v1 = pd.read_csv(cluster1)
df_cluster_v2 = pd.read_csv(cluster2)

In [4]:
df_cluster_v1.drop('Unnamed: 0', axis=1, inplace=True)
df_cluster_v2.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_cluster_v1.head()

,key,pid,size,Cluster,brand,brand_code,category,color,color_code,mainCategory,releaseDate,rrp,rrp_range,size_normalization,size_range,stock,subCategory,total_sales,total_sales_range,total_sales_range2
0,10000XL ( 158-170 ),10000.0,XL ( 158-170 ),226,Nike,0.0,1.428571,gruen,3.750,0.0,2017/10/1,25.33,0.0,XL,5.555556,0.0,5.365854,1.0,0.0,0.0
1,10001L,10001.0,L,0,Jako,6.0,1.428571,schwarz,8.125,0.0,2017/10/1,38.03,2.5,L,4.444444,0.0,3.170732,4.0,0.0,0.0
2,100033 (35-38 ),10003.0,3 (35-38 ),-1,Jako,6.0,1.428571,weiss,10.000,0.0,2017/10/1,12.63,0.0,M,3.333333,0.0,2.439024,15.0,0.0,0.0
3,100034 ( 39-42 ),10003.0,4 ( 39-42 ),86,Jako,6.0,1.428571,weiss,10.000,0.0,2017/10/1,12.63,0.0,L,4.444444,0.0,2.439024,3.0,0.0,0.0
4,100035 ( 43-46 ),10003.0,5 ( 43-46 ),1,Jako,6.0,1.428571,weiss,10.000,0.0,2017/10/1,12.63,0.0,XL,5.555556,0.0,2.439024,11.0,0.0,0.0


In [6]:
df_cluster_v2.head()

,key,pid,size,Cluster,brand,brand_code,category,color,color_code,mainCategory,releaseDate,rrp,rrp_range,size_normalization,size_range,stock,subCategory,total_sales,total_sales_range,total_sales_range2
0,10000XL ( 158-170 ),10000.0,XL ( 158-170 ),0,Nike,0.0,1.428571,gruen,3.750,0.0,2017/10/1,25.33,0.0,XL,5.555556,0.0,5.365854,1.0,0.0,0.0
1,10001L,10001.0,L,1,Jako,6.0,1.428571,schwarz,8.125,0.0,2017/10/1,38.03,2.5,L,4.444444,0.0,3.170732,4.0,0.0,0.0
2,100033 (35-38 ),10003.0,3 (35-38 ),2,Jako,6.0,1.428571,weiss,10.000,0.0,2017/10/1,12.63,0.0,M,3.333333,0.0,2.439024,15.0,0.0,0.0
3,100034 ( 39-42 ),10003.0,4 ( 39-42 ),3,Jako,6.0,1.428571,weiss,10.000,0.0,2017/10/1,12.63,0.0,L,4.444444,0.0,2.439024,3.0,0.0,0.0
4,100035 ( 43-46 ),10003.0,5 ( 43-46 ),4,Jako,6.0,1.428571,weiss,10.000,0.0,2017/10/1,12.63,0.0,XL,5.555556,0.0,2.439024,11.0,0.0,0.0


In [7]:
df_train = df.drop(df.tail(31).index)

In [8]:
df_train.tail()

,ds,10000XL ( 158-170 ),10001L,100033 (35-38 ),100034 ( 39-42 ),100035 ( 43-46 ),10006XL,10008XL,10013L,10013M,...,2286946,2286947,"2286947,5",22872M ( 140-152 ),22873L,228782XL,22878L,22878M,22878XL,22881S
87,2017-12-27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
88,2017-12-28,0,0,0,0,0,0,0,0,0,...,0,0,10,0,0,10,30,10,20,0
89,2017-12-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,2017-12-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,2017-12-31,0,0,0,0,0,0,0,0,0,...,0,0,0,30,0,0,0,0,0,10


In [9]:
df_ds = df_train.iloc[:,[0]]

In [10]:
df_ds.head()

,ds
0,2017-10-01
1,2017-10-02
2,2017-10-03
3,2017-10-04
4,2017-10-05


In [11]:
keys = df_cluster_v1['key'][df_cluster_v1['Cluster'] == 0]
#data['duration'][data['item'] == 'call'].sum()
print(df_train[keys])
print(df_train[keys].mean(axis=1))

    10001L  10084L  11706L  11768L  13919L  14461L  15613L  16342L  17101L  \
0        0       0      10       0       0       0       0       0       0   
1        0      10       0       0       0      10       0       0       0   
2        0      10       0       0       0       0       0       0       0   
3        0       0       0       0       0       0       0       0       0   
4        0      10      10       0       0       0      10       0      10   
5        0       0       0       0       0       0       0       0       0   
6        0       0       0       0       0       0       0       0       0   
7        0      20      10       0       0       0       0       0       0   
8        0       0      10      10       0       0       0       0       0   
9        0       0      20       0       0      10      40       0       0   
10       0       0      10       0       0       0      10       0       0   
11       0       0       0       0       0       0       0      

In [12]:
#Aggregate one df per Cluster
#Cluster v1 (200+ clusters)
list_of_dataframes_200 = []
for i in range(0, df_cluster_v1['Cluster'].max()):
    print(i)
    keys = df_cluster_v1['key'][df_cluster_v1['Cluster'] == i]
    df_mean = df_train[keys].mean(axis=1)
    df_to_be_appended = df_ds
    df_to_be_appended['y'] = df_mean
    
    df_to_be_appended.columns = ['ds', 'y']
    
    list_of_dataframes_200.append(df_to_be_appended)

0
1
2
3
4
5


C:\Users\Philipp\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


6
7
8
9
10
11
12


KeyError: "['2119439 715827883/2' '2119440 1431655765/2' '2119441 715827883/2'\n '2144540 1431655765/2' '2173841 715827883/2' '2217040 1431655765/2'\n '2217041 715827883/2' '2268541 715827883/2'] not in index"

In [13]:
#Cluster v2 (500+ clusters)
list_of_dataframes_500 = []
for i in range(1, df_cluster_v2['Cluster'].max()):
    #do sth
    print(i)
    keys = df_cluster_v1['key'][df_cluster_v1['Cluster'] == i]
    df_mean = df_train[keys].mean(axis=1)
    df_to_be_appended = df_ds
    df_to_be_appended['y'] = df_mean
    
    df_to_be_appended.columns = ['ds', 'y']
    
    list_of_dataframes_500.append(df_to_be_appended)

1
2
3
4
5


C:\Users\Philipp\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


6
7
8
9
10
11
12


KeyError: "['2119439 715827883/2' '2119440 1431655765/2' '2119441 715827883/2'\n '2144540 1431655765/2' '2173841 715827883/2' '2217040 1431655765/2'\n '2217041 715827883/2' '2268541 715827883/2'] not in index"

In [ ]:
#Forecasting
################################
# Taken from Basil: datamining2/data/datasets_Basil/create_dataset_v0.3.ipynb
#creating array with dates where 11 Teamsports announces discount on social media
marketingactivities=['2017-10-11','2017-10-16','2017-11-04','2017-11-11','2017-11-23','2017-11-24',
                     '2017-11-25','2017-11-27','2017-12-03','2017-12-27','2017-12-28','2017-12-31',
                    '2018-01-14','2018-01-22','2018-01-23','2018-01-30','2018-02-06','2018-02-07',
                     '2018-02-20','2018-02-22','2018-02-23']
################################

In [ ]:
marketingactivities_prophet = pd.DataFrame({
    'holiday': 'marketingactivity',
    'ds': pd.to_datetime(marketingactivities),
    })

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
prophet_fitted_200 = []
prophet_fitted_500 = []

In [ ]:
if read_write_prophet_fitted == 'w':
    #Cluster v1 with 200+ clusters
    for idx, val in enumerate(list_of_dataframes_200):
        prophet_fitted_200.append(Prophet(holidays = marketingactivities_prophet).fit(val))
    #Cluster v2 with 500+ clusters
    for idx, val in enumerate(list_of_dataframes_500):
        prophet_fitted_500.append(Prophet(holidays = marketingactivities_prophet).fit(val))

In [ ]:
if read_write_prophet_fitted == 'w':
    with open(output + "prophet_fitted_200", "wb") as fp:
        pickle.dump(prophet_fitted_200, fp)
    with open(output + "prophet_fitted_500", "wb") as fp:
        pickle.dump(prophet_fitted_500, fp)

In [ ]:
if read_write_prophet_fitted == 'r':
    with open(output + "prophet_fitted_200", "rb") as fp:
        prophet_fitted_200 = pickle.load(fp)
    with open(output + "prophet_fitted_500", "rb") as fp:
        prophet_fitted_500 = pickle.load(fp)

In [ ]:
prophet_forecasts_200 = []
prophet_forecasts_500 = []

In [ ]:
if read_write_prophet_forecasts == 'w':
    #As before, cluster v1
    for idx, val in enumerate(prophet_fitted_200):
        future = val.make_future_dataframe(periods=31)[92:]
        prophet_forecasts_200.append(val.predict(future)[['ds', 'yhat']])
    #As before, cluster v2
    for idx, val in enumerate(prophet_fitted_500):
        future = val.make_future_dataframe(periods=31)[92:]
        prophet_forecasts_500.append(val.predict(future)[['ds', 'yhat']])

In [ ]:
if read_write_prophet_forecasts == 'w':
    with open(output + "prophet_forecasts_200", "wb") as fp:
        pickle.dump(prophet_forecasts_200, fp)
    with open(output + "prophet_forecasts_500", "wb") as fp:
        pickle.dump(prophet_forecasts_500, fp)

In [ ]:
if read_write_prophet_forecasts == 'r':
    with open(output + "prophet_forecasts_200", "rb") as fp:
        prophet_forecasts_200 = pickle.load(fp)
    with open(output + "prophet_forecasts_500", "rb") as fp:
        prophet_forecasts_500 = pickle.load(fp)

In [ ]:
#Evaluation